In [1]:
from mpl_toolkits import mplot3d

import trimesh

%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import numpy as np
from scipy.io import loadmat

import os
import tqdm

from utils import coef2object, InvalidArgumentError, \
                default_render_params, create_render_params

MFM_DIR = "../PublicMM1/"
MFM_NAME = "01_MorphableModel.mat"

searching for scad in: /Users/gergelyflamich/Documents/Miscellaneous/MorphableFaceModel/mfm_venv/bin:/Users/gergelyflamich/Documents/Miscellaneous/MorphableFaceModel/mfm_venv/bin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/Applications/OpenSCAD.app/Contents/MacOS


In [3]:
# Load the MFM
if 'model' not in globals() or model is None:
    model = loadmat(MFM_DIR + MFM_NAME)
    model_size = {
        "n_shape_dim": model["shapePC"].shape[1],
        "n_tex_dim": model["texPC"].shape[1],
        "n_seg": model["segbin"].shape[1]
    }
    print("Loaded model!")
    
else:
    print("Model already loaded!")

Loaded model!


In [4]:
def display_face(shape, texture, triangulation, render_params):
    """
    Based on display_face.m provided in the University of Basel
    MFM package.
    """
   
    shape_size = np.prod(shape.shape)
    texture_size = np.prod(texture.shape)
    
    shape = shape.reshape((shape_size // 3, 3))
    texture = texture.reshape((texture_size // 3, 3))
    
    mesh = trimesh.Trimesh(vertices=shape,
                           faces=triangulation,
                           vertex_colors=texture)

    return mesh

In [9]:
alpha = np.zeros((model_size["n_shape_dim"], 1))
beta = np.zeros((model_size["n_tex_dim"], 1))

shape = coef2object(alpha, 
                    model["shapeMU"], 
                    model["shapePC"], 
                    model["shapeEV"])

texture = coef2object(beta, 
                      model["texMU"], 
                      model["texPC"], 
                      model["texEV"])

render_params = create_render_params(phi=0.5)

# The triangle indices are from Matlab, which means that they start at 1
# so we correct it
triangulation = model["tl"] - 1

# Quick fix to make the winding direction of the faces right
triangulation = triangulation[:, ::-1]

face = display_face(shape=shape, 
                     texture=texture, 
                     triangulation=triangulation, 
                     render_params=render_params)

face.show()

(160470, 1)
(106466, 3)
